# Model Context Protocol (MCP): Server Integration Fundamentals

This notebook introduces **MCP (Model Context Protocol)**, Anthropic's open standard for connecting AI applications to external tools and data sources. MCP provides:
1. **Standardized tool discovery**: Servers expose capabilities via a uniform interface
2. **Language-agnostic integration**: Python and JavaScript servers work seamlessly
3. **Ecosystem compatibility**: Growing marketplace of pre-built MCP servers

**Key Pattern:** MCP separates tool implementation (servers) from agent logic (clients), enabling plug-and-play tool integration.

In [ ]:
# Import dependencies
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

load_dotenv(override=True)

## Phase 1: Connect to MCP Fetch Server (Python)

The `mcp-server-fetch` provides web scraping capabilities.

In [ ]:
# Initialize MCP Server
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

## Phase 2: Connect to MCP Playwright Server (JavaScript/Node.js)

Browser automation via Playwright MCP server.

In [ ]:
# Playwright MCP Server
playwright_params = {"command": "npx", "args": ["@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools

## Phase 3: Connect to Filesystem MCP Server

In [ ]:
# Filesystem MCP Server (Sandboxed)
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {
    "command": "npx", 
    "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]
}

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

## Phase 4: Build Agent with Multiple MCP Servers

In [ ]:
# Agent with Browser + File System Tools
instructions = """You browse the internet to accomplish tasks.
You are highly capable at navigating websites independently, handling cookies, 
and finding content. If one site isn't helpful, try another. Be persistent."""

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="web_researcher", 
            instructions=instructions, 
            model="gpt-4o-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
        )
        
        with trace("research_task"):
            result = await Runner.run(
                agent, 
                "Find a great recipe for Banoffee Pie, then save it to banoffee.md"
            )
            print(result.final_output)

## MCP Ecosystem Resources

**MCP Marketplaces:**
- [mcp.so](https://mcp.so)
- [Glama AI MCP](https://glama.ai/mcp)
- [Smithery](https://smithery.ai/)
- [HuggingFace Top 11 MCP Libraries](https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries)
- [HuggingFace MCP Community Article](https://huggingface.co/blog/Kseniase/mcp)